# Initial setting

## libraries

In [1]:
#RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get(
    "https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css"
).text
HTML(styles)

In [2]:
import re
import requests
from time import sleep
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

from collections import OrderedDict

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

## dataset

In [3]:
# Load data
df = pd.read_csv('data/df_fifa.csv')

In [4]:
df.shape

(100995, 146)

In [5]:
df.head()

,sofifa_id,short_name,age,dob,height_cm,weight_kg,nationality,club,overall,value_eur,wage_eur,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,release_clause_eur,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,year,league_name,d_trait_Cautious_With_Crosses,d_trait_Injury_Prone,d_trait_Avoids_Using_Weaker_Foot,d_trait_Backs_Into_Player,d_trait_Flair,d_trait_Saves_with_Feet,d_trait_Leadership,d_trait_Inflexible,d_trait_Finesse_Shot,d_trait_Team_Player,d_trait_1-on-1_Rush,d_trait_Set_Play_Specialist,d_trait_Rushes_Out_Of_Goal,d_trait_Beat_Offside_Trap,d_trait_Early_Crosser,d_trait_Second_Wind,d_trait_Through_Ball,d_trait_Long_Throw-in,d_trait_Giant_Throw-in,d_trait_Diver,d_trait_Acrobatic_Clearance,d_trait_Selfish,d_trait_GK_Up_for_Corners,d_trait_Argues_with_Officials,d_trait_Takes_Finesse_Free_Kicks,...,d_pos_GK,ab_pace,ab_shooting,ab_passing,ab_dribbling,ab_defending,ab_physic,ab_skill_moves,ab_gk_diving,ab_gk_handling,ab_gk_kicking,ab_gk_reflexes,ab_gk_speed,ab_gk_positioning,ab_attacking_crossing,ab_attacking_finishing,ab_attacking_heading_accuracy,ab_attacking_short_passing,ab_attacking_volleys,ab_skill_dribbling,ab_skill_curve,ab_skill_fk_accuracy,ab_skill_long_passing,ab_skill_ball_control,ab_movement_acceleration,ab_movement_sprint_speed,ab_movement_agility,ab_movement_reactions,ab_movement_balance,ab_power_shot_power,ab_power_jumping,ab_power_stamina,ab_power_strength,ab_power_long_shots,ab_mentality_aggression,ab_mentality_interceptions,ab_mentality_positioning,ab_mentality_vision,ab_mentality_penalties,ab_mentality_composure,ab_defending_marking,ab_defending_standing_tackle,ab_defending_sliding_tackle,ab_goalkeeping_diving,ab_goalkeeping_handling,ab_goalkeeping_kicking,ab_goalkeeping_positioning,ab_goalkeeping_reflexes,d_foot_left,d_div1_league
0,158023,L. Messi,27,1987-06-24,169,67,Argentina,FC Barcelona,93,0,0,Left,5,3,4,Medium/Low,Normal,NaN,CF,10.0,NaN,2004-07-01,2018.0,2015,Spain Primera Division,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,93.0,89.0,86.0,96.0,27.0,63.0,4,NaN,NaN,NaN,NaN,NaN,NaN,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,1,1
1,20801,Cristiano Ronaldo,29,1985-02-05,185,80,Portugal,Real Madrid,92,0,0,Right,5,4,5,High/Low,Normal,NaN,LW,7.0,NaN,2009-07-01,2018.0,2015,Spain Primera Division,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,93.0,93.0,81.0,91.0,32.0,79.0,5,NaN,NaN,NaN,NaN,NaN,NaN,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,0,1
2,9014,A. Robben,30,1984-01-23,180,80,Netherlands,FC Bayern München,90,0,0,Left,5,2,4,High/Low,Normal,NaN,SUB,10.0,NaN,2009-08-28,2017.0,2015,German 1. Bundesliga,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,...,0,93.0,86.0,83.0,92.0,32.0,64.0,4,NaN,NaN,NaN,NaN,NaN,NaN,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,1,1
3,41236,Z. Ibrahimović,32,1981-10-03,195,95,Sweden,Paris Saint-Germain,90,0,0,Right,5,4,4,Medium/Low,Normal,NaN,ST,10.0,NaN,2012-07-01,2016.0,2015,French Ligue 1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,76.0,91.0,81.0,86.0,34.0,86.0,4,NaN,NaN,NaN,NaN,NaN,NaN,76,91,76,84,92,88,80,80,76,90,74,77,86,85,41,93,72,78,93,88,84,20,86,83,91,NaN,25,41,27,13,15,10,9,12,0,1
4,167495,M. Neuer,28,1986-03-27,193,92,Germany,FC Bayern München,90,0,0,Right,5,4,1,Medium/Medium,Normal,NaN,GK,1.0,NaN,2011-07-01,2019.0,2015,German 1. Bundesliga,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,1,87.0,85.0,92.0,86.0,60.0,90.0,25,25,25,42,25,25,25,25,41,31,58,61,43,89,35,42,78,44,83,25,29,30,25,20,37,NaN,25,25,25,87,85,92,90,86,0,1


# PartD

The overall approach for this part is as follows:
1. Create skill_summary column by taking a mean of all skill stats to use it as response variable
1. Create 5 different models using below as a training data set: 1) predicting 2016 from 2015 dataset, 2) predicting 2017 from 2016 dataset, 3) predicting 2018 from 2017 dataset, 4) predicting 2019 from 2018 dataset, 5) predicting 2020 from 2019 dataset.
1. Use above 5 models (XGBoost) and predict 2021 skill_summary stats from 2020 dataset.
1. Using the weight of 1) 0.15, 2)0.15, 3)0.20, 4)0.20, 5)0.30 to ensemble the models and make prediction. The weight is decided according to the fact that in the test to predict 2020 summary stats, the models trained by recent data had lower MSE score.

In [6]:
# create a list of skills and create skill_summary column by taking mean of them.
skills = []
for i in list(df.columns):
    if 'skill' in i:
        skills.append(i)
df['skill_summary'] = (df['ab_skill_dribbling'] + df['ab_skill_curve'] + df['ab_skill_fk_accuracy'] + df['ab_skill_long_passing'] + df['ab_skill_ball_control']) / 5

## Train 3 models using each year's dataset to predict skill_summary

### XGBOOST

In [7]:
# import packages
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [8]:
def partd_xgb(train_on, validate_on):
    # preprocessing
    # subset based on the inputed year
    df_tr = df[df['year']==train_on]
    df_val = df[df['year']==validate_on]

    # drop variables other than 'skill_summary'
    df_val_sk = df_val[['sofifa_id', 'skill_summary']]

    # merge and drop unnecessary columns/rows, imputation
    df_merge = df_tr.merge(df_val_sk, on='sofifa_id', how='left', suffixes=(None, '_r'))
    df_merge.dropna(subset=['skill_summary_r'], inplace=True)
    drop_vars = ['sofifa_id','short_name','dob','nationality','work_rate','body_type','team_position','loaned_from',
                'preferred_foot','joined','contract_valid_until','year','main_position','team_jersey_number', 'wage_eur', 'club', 'ab_mentality_composure']
    df_merge_drop = df_merge.drop(drop_vars, axis=1)
    df_merge_drop = df_merge_drop[-df_merge_drop.ab_pace.isna()]
    impute_vars = ['release_clause_eur'] + [x for x in df_merge.columns if x.startswith('ab_gk')]
    for var in impute_vars:
        df_merge_drop[var] = df_merge_drop[var].fillna(0)

    # select players in the following clubs as per instruction
    select_clubs = [ 'English Premier League',  'German 1. Bundesliga',  'French Ligue 1', 'Spain Primera Division', 'Italian Serie A']
    df_selected = df_merge_drop[df_merge_drop.league_name.isin(select_clubs)].drop(['league_name'], axis=1)

    # separate X and y
    y = df_selected['skill_summary_r']
    X = df_selected.drop(['skill_summary_r'], axis=1)

    # random search
    param_space = {
        'min_child_weight': hp.loguniform('min_child_weight', np.log(1), np.log(10)),
        'max_depth': hp.quniform('max_depth', 3, 9, 1),
        'subsample': hp.quniform('subsample', 0.6, 0.95, 0.05),
        'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.95, 0.05),
        'gamma': hp.loguniform('gamma', np.log(1e-8), np.log(1.0)),
        'reg_alpha': hp.loguniform('reg_alpha', np.log(1e-8), np.log(1.0)),
        'reg_lambda': hp.loguniform('reg_lambda', np.log(1e-6), np.log(10.0))
    #     'booster': hp.choice('booster', ['gblinear', 'dart', 'gbtree'])
    }

    def score(params):
        xgb = XGBRegressor(random_state=81, min_child_weight=params['min_child_weight'],
                          max_depth=int(params['max_depth']), subsample=params['subsample'], colsample_bytree=params['colsample_bytree'],
                          gamma=params['gamma'], reg_alpha =params['reg_alpha'], reg_lambda =params['reg_lambda']) #, booster=params['booster']
        scores = cross_validate(xgb, X=X, y=y, cv=5, scoring='neg_mean_squared_error')
        return -1 * scores['test_score'].mean()

    max_evals = 100
    trials = Trials()
    history = []
    rstate = np.random.RandomState(81)
    best = fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals, rstate=rstate)

    # fit best model
    best_model = XGBRegressor(random_state=81, min_child_weight=best['min_child_weight'],
                          max_depth=int(best['max_depth']), subsample=best['subsample'], colsample_bytree=best['colsample_bytree'],
                          gamma=best['gamma'], reg_alpha =best['reg_alpha'], reg_lambda =best['reg_lambda']).fit(X, y)
    return best, best_model

In [9]:
def test_function(model):
    # get 2019 and 2020 data for training
    df_tr = df[df['year']==2019]
    df_val = df[df['year']==2020]

    # drop variables other than 'skill_summary' from df_2020
    df_val_sk = df_val[['sofifa_id', 'skill_summary']]

    # merge with df_2019 and drop unnecessary columns and rows
    df_merge = df_tr.merge(df_val_sk, on='sofifa_id', how='left', suffixes=(None, '_r'))
    df_merge.dropna(subset=['skill_summary_r'], inplace=True)
    drop_vars = ['sofifa_id','short_name','dob','nationality','work_rate','body_type','team_position','loaned_from',
                'preferred_foot','joined','contract_valid_until','year','main_position','team_jersey_number', 'wage_eur', 'club', 'ab_mentality_composure']
    df_merge_drop = df_merge.drop(drop_vars, axis=1)
    df_merge_drop = df_merge_drop[-df_merge_drop.ab_pace.isna()]

    # impute zero values for release_clause and goalkeeping ability
    # this is because goalkeeping ability is not available for field players
    impute_vars = ['release_clause_eur'] + [x for x in df_merge.columns if x.startswith('ab_gk')]
    for var in impute_vars:
        df_merge_drop[var] = df_merge_drop[var].fillna(0)

    # select players in the following clubs as test set
    select_clubs = [ 'English Premier League',  'German 1. Bundesliga',  'French Ligue 1', 'Spain Primera Division', 'Italian Serie A']
    df_selected = df_merge_drop[df_merge_drop.league_name.isin(select_clubs)].drop(['league_name'], axis=1)
    print(df_selected.shape)

    # separate X and y
    y = df_selected['skill_summary_r']
    X = df_selected.drop(['skill_summary_r'], axis=1)
    
    te_pred = model.predict(X)
    test_score = mean_squared_error(y, te_pred)

    return test_score

In [10]:
result_xgb = {}
model_xgb = {}
for i in [2015, 2016, 2017, 2018, 2019]:
    best_param, best_model = partd_xgb(i, i+1)
    test_score = test_function(best_model)
    result_xgb[i] = test_score
    model_xgb[i] = best_model

100%|██████████| 100/100 [03:11<00:00,  1.91s/trial, best loss: 11.013538135760598]
(2329, 130)
100%|██████████| 100/100 [03:17<00:00,  1.98s/trial, best loss: 8.13766804832419]
(2329, 130)
100%|██████████| 100/100 [03:51<00:00,  2.32s/trial, best loss: 8.683301112760518]
(2329, 130)
100%|██████████| 100/100 [03:21<00:00,  2.02s/trial, best loss: 7.2199835043737295]
(2329, 130)
100%|██████████| 100/100 [03:46<00:00,  2.26s/trial, best loss: 6.973173504432326]
(2329, 130)


In [11]:
print(f'The MSE on 2020 skill_summary prediction by XGBoost is 2015 model: {result_xgb[2015]:.3f}, 2016 model: {result_xgb[2016]:.3f}, 2017 model: {result_xgb[2017]:.3f}, 2018 model: {result_xgb[2018]:.3f}, 2019 model: {result_xgb[2019]:.3f}.')

The MSE on 2020 skill_summary prediction by XGBoost is 2015 model: 8.182, 2016 model: 6.649, 2017 model: 6.817, 2018 model: 6.594, 2019 model: 2.215.


### Random Forest model

In [12]:
# same code as above, but with Random Forest
def partd_rf(train_on, validate_on):
    # preprocessing
    # subset based on the inputed year
    df_tr = df[df['year']==train_on]
    df_val = df[df['year']==validate_on]

    # drop variables other than 'skill_summary'
    df_val_sk = df_val[['sofifa_id', 'skill_summary']]

    # merge and drop unnecessary columns/rows, imputation
    df_merge = df_tr.merge(df_val_sk, on='sofifa_id', how='left', suffixes=(None, '_r'))
    df_merge.dropna(subset=['skill_summary_r'], inplace=True)
    drop_vars = ['sofifa_id','short_name','dob','nationality','work_rate','body_type','team_position','loaned_from',
                'preferred_foot','joined','contract_valid_until','year','main_position','team_jersey_number', 'wage_eur', 'club', 'ab_mentality_composure']
    df_merge_drop = df_merge.drop(drop_vars, axis=1)
    df_merge_drop = df_merge_drop[-df_merge_drop.ab_pace.isna()]
    impute_vars = ['release_clause_eur'] + [x for x in df_merge.columns if x.startswith('ab_gk')]
    for var in impute_vars:
        df_merge_drop[var] = df_merge_drop[var].fillna(0)

    # select players in the following clubs as per instruction
    select_clubs = [ 'English Premier League',  'German 1. Bundesliga',  'French Ligue 1', 'Spain Primera Division', 'Italian Serie A']
    df_selected = df_merge_drop[df_merge_drop.league_name.isin(select_clubs)].drop(['league_name'], axis=1)

    # separate X and y
    y = df_selected['skill_summary_r']
    X = df_selected.drop(['skill_summary_r'], axis=1)

    # random search
    param_space = {
        'max_depth': hp.quniform('max_depth', 3, 50, 1),
        'n_estimators': hp.quniform('n_estimators', 100, 500, 5)
#         'max_features': hp.choice('max_features', [int(X.shape[1]/3), int(X.shape[1])])
    }

    def score(params):
        depth = int(params['max_depth'])
        est = int(params['n_estimators'])
#         mf = int(params['max_features'])

        rf = RandomForestRegressor(random_state=81, n_estimators=est, max_depth=depth, max_features=int(X.shape[1]/3))
        scores = cross_validate(rf, X=X, y=y, cv=5, scoring='neg_mean_squared_error')
        return -1 * scores['test_score'].mean()

    max_evals = 10
    trials = Trials()
    history = []
    rstate = np.random.RandomState(81)
    best = fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals, rstate=rstate)

    # fit best model
    best_model = RandomForestRegressor(max_depth=int(best['max_depth']), n_estimators=int(best['n_estimators']), max_features=int(X.shape[1]/3), random_state=81).fit(X, y)
    return best, best_model

In [13]:
result_rf = {}
model_rf = {}
for i in [2015, 2016, 2017, 2018, 2019]:
    best_param, best_model = partd_rf(i, i+1)
    test_score = test_function(best_model)
    result_rf[i] = test_score
    model_rf[i] = best_model

100%|██████████| 10/10 [02:34<00:00, 15.45s/trial, best loss: 9.568520983471064]
(2329, 130)
100%|██████████| 10/10 [02:41<00:00, 16.13s/trial, best loss: 7.607957348938635]
(2329, 130)
100%|██████████| 10/10 [03:01<00:00, 18.13s/trial, best loss: 7.833635369475918]
(2329, 130)
100%|██████████| 10/10 [03:06<00:00, 18.62s/trial, best loss: 6.9752892194283564]
(2329, 130)
100%|██████████| 10/10 [03:13<00:00, 19.39s/trial, best loss: 6.367682390394821]
(2329, 130)


In [14]:
print(f'The MSE of Random Forest is 2015 model: {result_rf[2015]:.3f}, 2016 model: {result_rf[2016]:.3f}, 2017 model: {result_rf[2017]:.3f}, 2018 model: {result_rf[2018]:.3f}, 2019 model: {result_rf[2019]:.3f}.')

The MSE of Random Forest is 2015 model: 6.542, 2016 model: 5.730, 2017 model: 5.773, 2018 model: 5.640, 2019 model: 0.808.


### Linear Regression - Elastic Net

In [15]:
from sklearn.linear_model import ElasticNet

In [16]:
# same code as above, but with Random Forest
def partd_en(train_on, validate_on):
    # preprocessing
    # subset based on the inputed year
    df_tr = df[df['year']==train_on]
    df_val = df[df['year']==validate_on]

    # drop variables other than 'skill_summary'
    df_val_sk = df_val[['sofifa_id', 'skill_summary']]

    # merge and drop unnecessary columns/rows, imputation
    df_merge = df_tr.merge(df_val_sk, on='sofifa_id', how='left', suffixes=(None, '_r'))
    df_merge.dropna(subset=['skill_summary_r'], inplace=True)
    drop_vars = ['sofifa_id','short_name','dob','nationality','work_rate','body_type','team_position','loaned_from',
                'preferred_foot','joined','contract_valid_until','year','main_position','team_jersey_number', 'wage_eur', 'club', 'ab_mentality_composure']
    df_merge_drop = df_merge.drop(drop_vars, axis=1)
    df_merge_drop = df_merge_drop[-df_merge_drop.ab_pace.isna()]
    impute_vars = ['release_clause_eur'] + [x for x in df_merge.columns if x.startswith('ab_gk')]
    for var in impute_vars:
        df_merge_drop[var] = df_merge_drop[var].fillna(0)

    # select players in the following clubs as per instruction
    select_clubs = [ 'English Premier League',  'German 1. Bundesliga',  'French Ligue 1', 'Spain Primera Division', 'Italian Serie A']
    df_selected = df_merge_drop[df_merge_drop.league_name.isin(select_clubs)].drop(['league_name'], axis=1)

    # separate X and y
    y = df_selected['skill_summary_r']
    X = df_selected.drop(['skill_summary_r'], axis=1)

    # random search
    param_space = {
        'alpha': hp.uniform('alpha', 0.1, 1)
    }

    def score(params):
        alpha = (params['alpha'])

        en = ElasticNet(random_state=81, alpha=alpha, max_iter=100000)
        scores = cross_validate(en, X=X, y=y, cv=5, scoring='neg_mean_squared_error')
        return -1 * scores['test_score'].mean()

    max_evals = 10
    trials = Trials()
    history = []
    rstate = np.random.RandomState(81)
    best = fmin(score, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals, rstate=rstate)

    # fit best model
    best_model = ElasticNet(random_state=81, alpha=best['alpha'], max_iter=100000).fit(X, y)
    return best, best_model

In [17]:
result_el = {}
model_el = {}
for i in [2015, 2016, 2017, 2018, 2019]:
    best_param, best_model = partd_en(i, i+1)
    test_score = test_function(best_model)
    result_el[i] = test_score
    model_el[i] = best_model

100%|██████████| 10/10 [00:04<00:00,  2.20trial/s, best loss: 8.55522021723896]
(2329, 130)
100%|██████████| 10/10 [00:23<00:00,  2.35s/trial, best loss: 6.763071825559851]
(2329, 130)
100%|██████████| 10/10 [00:27<00:00,  2.78s/trial, best loss: 8.309706519528643]
(2329, 130)
100%|██████████| 10/10 [00:29<00:00,  2.95s/trial, best loss: 6.067784996595809]
(2329, 130)
100%|██████████| 10/10 [00:25<00:00,  2.50s/trial, best loss: 5.4282548351093025]
(2329, 130)


In [18]:
print(f'The MSE by Elastic Net model: 2015 model: {result_el[2015]:.3f}, 2016 model: {result_el[2016]:.3f}, 2017 model: {result_el[2017]:.3f}, 2018 model: {result_el[2018]:.3f}, 2019 model: {result_el[2019]:.3f}.')

The MSE by Elastic Net model: 2015 model: 6.062, 2016 model: 5.502, 2017 model: 5.457, 2018 model: 5.391, 2019 model: 5.221.


In [48]:
# dataframe to summarize all test result
df_result = pd.DataFrame({'xgb': result_xgb, 'rf': result_rf, 'el': result_el}).T
df_result['average'] = (df_result[2015] + df_result[2016] + df_result[2017] + df_result[2017] + df_result[2019]) / 5
display(df_result)

,2015,2016,2017,2018,2019,average
xgb,8.182012,6.648985,6.816944,6.593935,2.214559,6.135889
rf,6.541896,5.729842,5.772701,5.639941,0.807616,4.924951
el,6.062420,5.501702,5.457146,5.391069,5.221469,5.539977


## predict 2021 ball_control skill score from 2020 data using ensemble model

In [20]:
df_2020 = df[df['year'] == 2020]

In [21]:
# preprocess 2020 data
# drop unnecessary columns
drop_vars = ['sofifa_id','short_name','dob','nationality','work_rate','body_type','team_position','loaned_from',
            'preferred_foot','joined','contract_valid_until','year','main_position','team_jersey_number', 'wage_eur', 'club', 'ab_mentality_composure']
df_2020_drop = df_2020.drop(drop_vars, axis=1)
df_2020_drop = df_2020_drop[-df_2020_drop.ab_pace.isna()]

# impute zero values for release_clause and goalkeeping ability
# this is because goalkeeping ability is not available for field players
impute_vars = ['release_clause_eur'] + [x for x in df_2020.columns if x.startswith('ab_gk')]
for var in impute_vars:
    df_2020_drop[var] = df_2020_drop[var].fillna(0)

# select players in the following clubs as test set
select_clubs = [ 'English Premier League',  'German 1. Bundesliga',  'French Ligue 1', 'Spain Primera Division', 'Italian Serie A']
df_2020_test = df_2020_drop[df_2020_drop.league_name.isin(select_clubs)].drop(['league_name'], axis=1)
print(df_2020_test.shape)

(2708, 129)


In [22]:
# making prediction for 2021 skill score
prediction = {}

for i in [2015, 2016, 2017, 2018, 2019]:
    for num, model in enumerate([model_xgb, model_rf, model_el]):
        model_pred = model[i]
        prediction_2021 = model_pred.predict(df_2020_test)
        prediction[i, num] = prediction_2021        
df_ensemble = pd.DataFrame(prediction)

# apply weight (please see presentation for detail)
df_ensemble['final_prediction'] = df_ensemble[2015][0] * 0.0375 + df_ensemble[2015][1] * 0.06 + df_ensemble[2015][2] * 0.0525 + df_ensemble[2016][0] * 0.0375 + df_ensemble[2016][1] * 0.06 + df_ensemble[2016][2] * 0.0525 + df_ensemble[2017][0] * 0.05 + df_ensemble[2017][1] * 0.08 + df_ensemble[2017][2] * 0.07 + df_ensemble[2018][0] * 0.05 + df_ensemble[2018][1] * 0.08 + df_ensemble[2018][2] * 0.07 + df_ensemble[2019][0] * 0.075 + df_ensemble[2019][1] * 0.12 + df_ensemble[2019][2] * 0.105

In [46]:
# integrate sofifa_id to the final dataframe for easy grading

# drop unnecessary columns, keep sofifa_id
drop_vars = ['short_name','dob','nationality','work_rate','body_type','team_position','loaned_from',
            'preferred_foot','joined','contract_valid_until','year','main_position','team_jersey_number', 'wage_eur', 'club', 'ab_mentality_composure']
df_2020_drop = df_2020.drop(drop_vars, axis=1)
df_2020_drop = df_2020_drop[-df_2020_drop.ab_pace.isna()]

# impute zero values for release_clause and goalkeeping ability
# this is because goalkeeping ability is not available for field players
impute_vars = ['release_clause_eur'] + [x for x in df_2020.columns if x.startswith('ab_gk')]
for var in impute_vars:
    df_2020_drop[var] = df_2020_drop[var].fillna(0)

# select players in the following clubs as test set
select_clubs = [ 'English Premier League',  'German 1. Bundesliga',  'French Ligue 1', 'Spain Primera Division', 'Italian Serie A']
df_2020_test = df_2020_drop[df_2020_drop.league_name.isin(select_clubs)].drop(['league_name'], axis=1)
df_id = df_2020_test['sofifa_id']
print(df_id.shape)

df_id_new = pd.DataFrame(df_id).reset_index()
df_ensemble['sofifa_id'] = df_id_new['sofifa_id']

(2708,)


In [47]:
# export in csv
df_ensemble.to_csv('final_prediction_partd.csv')